# PAGE - CONVENTION COLLECTIVE

**Définition** : 

Ce notebook permet de générer les KPIs autours de la page des conventions collectives :

 * Convertion des conventions collectives : Taux de visites où l'utilisateur a cliqué sur une contribution


## Récupération des données

In [ ]:
import pandas as pd
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from src.elasticsearch_connector import ElasticsearchConnector
#from typing import List, Dict


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

### Interval

Renseignez la date de début (incluse) et de fin (non incluse) souhaitées pour la récupération des données

In [ ]:
date_debut = '2024-11-01'
date_fin = '2024-11-30'

### Convention collectives supportées

Utilisé pour filtrer dans la requête ES et allégé le processus

In [ ]:
cc_ids = ['2511','1596','1597','3127','1404','3043','1517','1527','2941','1043','2609','2596','2614','1483','3239','2264','2216','2148','2098','2120','1996','1979','1702','1672','1516','1518','1606','1501','1505','1486','1480','1351','1266','1147','1090','843','675','292','275','176','86','44','16','29','3248','2420','413','787','573']  # Liste des identifiants des conventions collectives

### Requête ES

In [ ]:
es_connector = ElasticsearchConnector(env='monolog')

# Construire dynamiquement les conditions sur les URLs
convention_clauses = [
    {"prefix": {"url": f"https://code.travail.gouv.fr/convention-collective/{cc_id}-"}} for cc_id in cc_ids
]

# Construire dynamiquement les conditions sur les URLs des contributions
contribution_clauses = [
    {"prefix": {"url": f"https://code.travail.gouv.fr/contribution/{cc_id}-"}} for cc_id in cc_ids
]

# Combiner les clauses dans le should
should_clauses = convention_clauses + contribution_clauses

QUERY = {
  "query": {
    "bool": {
      "must": [
        {
          "bool": {
            "should": should_clauses,  # Liste dynamique des préfixes pour conventions et contributions
            "minimum_should_match": 1
          }
        },
        {
          "range": {
            "logfile": {
              "gte": date_debut,
              "lt": date_fin
            }
          }
        },
        {
          "terms": {
            "type": [
              "visit_content"
            ]
          }
        }
      ]
    }
  }
}

In [ ]:
inital_logs = es_connector.execute_query(QUERY, "logs-new")

## Taux de visites où l'utilisateur a cliqué sur une contribution

### Calcul du nombre de visites sur les conventions collectives ayant abouti à un click sur une contribution

Pour ce calcul, on va prendre toutes les visites sur les pages de conventions collectives qui ont également une visites sur les contributions personnalisées. 

Note:
Afin de simplifier le processus et éviter de récupérer l'ensemble des events depuis ES, on va vérifier que la contribution a été visité dans un délai de moins de 2 minutes. Cela permet d'exclure les visites où l'utilisateur arrive sur une contribution mais pas directement depuis la convention collective.
Ce qui serait top, c'est de récupérer tout les events depuis ES, de prendre les visites où l'utilisateur est passé sur une convention collective puis direct après sur une contribution (et en plus sur une contribution de la même convention collective). Cela demande beaucoup d'effort et d'énergie pour le moment.

In [ ]:
logs = inital_logs.copy()

# Convertir timestamp en datetime pour trier facilement
logs['date'] = pd.to_datetime(logs['timestamp'], unit='s')

# Trier par idVisit et timestamp
sorted_logs = logs.sort_values(by=['idVisit', 'date'])

def filter_visits(group):
    urls = group['url'].tolist()
    timestamps = group['date'].tolist()
    
    # Vérifier qu'il y a au moins deux URLs pour comparer
    if len(urls) > 1:
        # Vérifier que la première URL est une convention collective
        if urls[0].startswith("https://code.travail.gouv.fr/convention-collective/"):
            # Vérifier que la deuxième URL est une contribution
            if urls[1].startswith("https://code.travail.gouv.fr/contribution/"):
                # Calculer la différence de temps (en secondes)
                time_diff = (timestamps[1] - timestamps[0]).total_seconds()
                if time_diff <= 120:  # 2 minutes = 120 secondes
                    return True
    return False

# Appliquer le filtre sur les groupes
grouped_logs = sorted_logs.groupby('idVisit')
valid_visits = grouped_logs.filter(filter_visits)

# Extraire l'identifiant de la convention collective de l'URL
valid_visits['convention_id'] = valid_visits['url'].str.extract(r"convention-collective/(\d+)")

# Filtrer les visites contenant uniquement les conventions collectives
convention_visits = valid_visits.dropna(subset=['convention_id'])

# Compter les visites uniques par convention collective
unique_visits = convention_visits.groupby('convention_id')['idVisit'].nunique().reset_index()
unique_visits.rename(columns={'idVisit': 'unique_visits'}, inplace=True)

### Calcul du nombre de visites par convention collective

In [ ]:
visits_per_cc = inital_logs.copy()

# Convertir le timestamp (en secondes) en datetime
visits_per_cc['timestamp'] = pd.to_datetime(visits_per_cc['timestamp'], unit='s')

# Extraire l'identifiant de la convention collective
visits_per_cc['convention_id'] = visits_per_cc['url'].str.extract(r"convention-collective/(\d+)")

# Filtrer les événements liés aux conventions collectives
cc_visits = visits_per_cc.dropna(subset=['convention_id'])

# Calculer le nombre de visites uniques pour chaque convention collective
unique_visits_cc = cc_visits.groupby('convention_id')['idVisit'].nunique().reset_index()
unique_visits_cc.rename(columns={'idVisit': 'unique_visits'}, inplace=True)

### Ratio du nombre de visites avec convertion pour chaque convention collective

In [ ]:
# Renommer les colonnes pour éviter les conflits
unique_visits.rename(columns={"unique_visits": "visits_converted"}, inplace=True)
unique_visits_cc.rename(columns={"unique_visits": "visits_total"}, inplace=True)

# Fusionner les deux DataFrames sur convention_id
merged_df = pd.merge(unique_visits, unique_visits_cc, on="convention_id", how="inner")

# Calculer le ratio
merged_df['ratio'] = merged_df['visits_converted'] / merged_df['visits_total'] * 100

In [ ]:
# Trie par ratio
merged_df = merged_df.sort_values(by='visits_total', ascending=False)

styled_df = (
    merged_df.style
    .format({"ratio": "{:.2f}%"})
)

styled_df

### Sauvegarde du tableau dans un fichier CSV

Le fichier est disponible dans le dossier `explorations/outputs`

In [ ]:
merged_df.rename(columns={
    'convention_id': 'CC',
    'visits_converted': 'Visites contrib',
    'visits_total': 'Visites total',
    'ratio': 'ratio'
}, inplace=True)

# Réorganiser l'ordre des colonnes
merged_df = merged_df[['CC', 'Visites total', 'Visites contrib', 'ratio']]

merged_df.to_csv(f"./outputs/ratio_completion_convention_collective_{date_debut}_{date_fin}.csv",  index=False, encoding='utf-8')